In [2]:
# GPU Memory Management
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear GPU cache
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
print("✅ GPU memory optimizations set")


✅ GPU memory optimizations set


In [3]:
# Install required packages including PDF processing
!pip install rouge-score transformers torch sentencepiece nltk sacrebleu bert-score kagglehub PyPDF2 pdfplumber -q

print("✅ All packages installed successfully!")


✅ All packages installed successfully!


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import glob
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModel
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# PDF processing
import PyPDF2
import pdfplumber
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All imports successful!")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ All imports successful!


In [5]:
import os
import glob
import random

# Dataset path (already added as input in Kaggle)
path = '/kaggle/input/legal-dataset-sc-judgments-india-19502024'

# Use 2025 folder only
year_folder = '2025'
final_path = os.path.join(path, 'supreme_court_judgments', year_folder)

print(f"📁 Path to dataset files: {path}")
print(f"🎯 Using year folder: {year_folder}")
print(f"📂 Final path: {final_path}")


📁 Path to dataset files: /kaggle/input/legal-dataset-sc-judgments-india-19502024
🎯 Using year folder: 2025
📂 Final path: /kaggle/input/legal-dataset-sc-judgments-india-19502024/supreme_court_judgments/2025


In [6]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF with multiple fallback methods"""
    text = ""
    
    try:
        # Method 1: pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        
        if text.strip():
            return clean_pdf_text(text)
    except Exception as e:
        pass
    
    try:
        # Method 2: PyPDF2 (fallback)
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        
        if text.strip():
            return clean_pdf_text(text)
    except Exception as e:
        pass
    
    return None

def clean_pdf_text(text):
    """Clean extracted PDF text"""
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove page numbers
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
    text = re.sub(r'Page \d+ of \d+', '', text, flags=re.IGNORECASE)
    
    # Remove headers/footers
    text = re.sub(r'Supreme Court of India.*?\n', '', text, flags=re.IGNORECASE)
    
    # Fix common PDF issues
    text = text.replace('�', '')
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r'\n+', '\n', text)
    
    return text.strip()

print("✅ PDF processing utilities defined")


✅ PDF processing utilities defined


In [7]:
class PDFLegalDataset(Dataset):
    """
    Dataset for PDF-based Supreme Court judgments
    FIXED: Handles both .pdf and .PDF extensions
    """
    def __init__(self, pdf_source, min_words=50):
        self.aspects = ['facts', 'analysis', 'argument', 'judgement', 'statute']
        
        # Handle both directory and list of paths
        if isinstance(pdf_source, str):
            # Search for both .pdf and .PDF (case-insensitive)
            pdf_lower = glob.glob(os.path.join(pdf_source, '**/*.pdf'), recursive=True)
            pdf_upper = glob.glob(os.path.join(pdf_source, '**/*.PDF'), recursive=True)
            self.pdf_files = sorted(pdf_lower + pdf_upper)
            print(f"📁 Loading from directory: {pdf_source}")
        elif isinstance(pdf_source, list):
            self.pdf_files = pdf_source
            print(f"📁 Loading from provided list of {len(pdf_source)} PDFs")
        else:
            raise ValueError("pdf_source must be either a directory path (str) or list of PDF paths")
        
        print(f"   Found {len(self.pdf_files)} PDF files")
        print(f"   Processing ALL PDFs (no limits)...\n")
        
        self.data = []
        failed_files = []
        
        for pdf_file in tqdm(self.pdf_files, desc="Extracting PDFs"):
            try:
                judgment_text = extract_text_from_pdf(pdf_file)
                
                if judgment_text and len(judgment_text.split()) >= min_words:
                    self.data.append({
                        'judgment': judgment_text,
                        'judgment_file': os.path.basename(pdf_file),
                        'pdf_path': pdf_file
                    })
                else:
                    failed_files.append(os.path.basename(pdf_file))
            except Exception as e:
                failed_files.append(os.path.basename(pdf_file))
        
        print(f"\n📊 Dataset Statistics:")
        print(f"   ✅ Successfully loaded: {len(self.data)} judgments")
        print(f"   ❌ Failed/too short: {len(failed_files)} files")
        
        if self.data:
            word_counts = [len(item['judgment'].split()) for item in self.data]
            sentence_counts = [len(sent_tokenize(item['judgment'])) for item in self.data]
            print(f"\n   📈 Document Statistics:")
            print(f"     Words - Min: {min(word_counts):,} | Max: {max(word_counts):,} | Avg: {int(np.mean(word_counts)):,}")
            print(f"     Sentences - Min: {min(sentence_counts):,} | Max: {max(sentence_counts):,} | Avg: {int(np.mean(sentence_counts)):,}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

print("✅ PDF Legal Dataset defined (supports .pdf and .PDF)")


✅ PDF Legal Dataset defined (supports .pdf and .PDF)


In [8]:
# Load dataset - use 'pdf_source' parameter (not 'pdf_dir')
dataset = PDFLegalDataset(
    pdf_source=final_path,  # Changed from pdf_dir to pdf_source
    min_words=50
)

if len(dataset) == 0:
    print("\n❌ No valid PDFs found!")
else:
    # 3-WAY SPLIT: Train (70%) / Validation (15%) / Test (15%)
    train_size = int(0.70 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size]
    )
    
    print(f"\n📈 3-WAY DATASET SPLIT:")
    print(f"{'='*70}")
    print(f"   Total documents:     {len(dataset)}")
    print(f"   Training set:        {len(train_dataset)} ({len(train_dataset)/len(dataset)*100:.1f}%)")
    print(f"   Validation set:      {len(val_dataset)} ({len(val_dataset)/len(dataset)*100:.1f}%)")
    print(f"   Test set:            {len(test_dataset)} ({len(test_dataset)/len(dataset)*100:.1f}%)")
    print(f"{'='*70}")
    
    # Show sample
    print(f"\n📄 Sample Document:")
    print(f"   File: {dataset[0]['judgment_file']}")
    print(f"   Length: {len(dataset[0]['judgment'].split())} words")
    print(f"   Preview: {dataset[0]['judgment'][:300]}...")


📁 Loading from directory: /kaggle/input/legal-dataset-sc-judgments-india-19502024/supreme_court_judgments/2025
   Found 400 PDF files
   Processing ALL PDFs (no limits)...



Extracting PDFs: 100%|██████████| 400/400 [07:40<00:00,  1.15s/it]



📊 Dataset Statistics:
   ✅ Successfully loaded: 400 judgments
   ❌ Failed/too short: 0 files

   📈 Document Statistics:
     Words - Min: 584 | Max: 41,504 | Avg: 5,656
     Sentences - Min: 32 | Max: 1,874 | Avg: 244

📈 3-WAY DATASET SPLIT:
   Total documents:     400
   Training set:        280 (70.0%)
   Validation set:      60 (15.0%)
   Test set:            60 (15.0%)

📄 Sample Document:
   File: A_John_Kennedy_vs_The_State_Of_Tamil_Nadu_on_24_March_2025_1.PDF
   Length: 3381 words
   Preview: A John Kennedy vs The State Of Tamil Nadu on 24 March, 2025 A John Kennedy vs The State Of Tamil Nadu on 24 March, 2025 Author: Vikram Nath Bench: Vikram Nath 2025 INSC 443 REPORTABLE IN THE SUPREME COURT OF INDIA CIVIL APPELLATE JURISDICTION SLP(CIVIL) NO(S). 999-1001 OF 2025 A. JOHN KENNEDY ETC. …...


In [9]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


🔧 Using device: cuda
   GPU: Tesla T4
   Memory: 15.83 GB


In [10]:
class SentenceEncoder(nn.Module):
    def __init__(self, model_name='law-ai/InLegalBERT', hidden_dim=768):
        super(SentenceEncoder, self).__init__()
        print(f"🔄 Loading {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.hidden_dim = hidden_dim
        
        # Freeze encoder
        for param in self.model.parameters():
            param.requires_grad = False
        
        print(f"✅ {model_name} loaded successfully (hidden_dim={hidden_dim})")
    
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def forward(self, sentences):
        encoded = self.tokenizer(sentences, padding=True, truncation=True, 
                                max_length=512, return_tensors='pt')
        encoded = {k: v.to(device) for k, v in encoded.items()}
        
        with torch.no_grad():
            model_output = self.model(**encoded)
        
        embeddings = self.mean_pooling(model_output, encoded['attention_mask'])
        return embeddings

print("✅ InLegalBERT Sentence Encoder defined")


✅ InLegalBERT Sentence Encoder defined


In [11]:
class MultiAspectPolicyNetwork(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256, num_aspects=5, dropout=0.5):
        super(MultiAspectPolicyNetwork, self).__init__()
        
        self.num_aspects = num_aspects
        self.aspects = ['facts', 'analysis', 'argument', 'judgement', 'statute']
        self.hidden_dim = hidden_dim
        
        # Shared LSTM encoder
        self.shared_lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2, 
                                   bidirectional=True, batch_first=True, dropout=dropout)
        
        # Position and aspect embeddings
        self.position_embedding = nn.Embedding(2000, 64)  # Increased for long documents
        self.aspect_embedding = nn.Embedding(num_aspects, hidden_dim * 2)
        
        # Attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,
            num_heads=4, 
            dropout=dropout,
            batch_first=True
        )
        
        # Aspect-specific heads
        self.aspect_heads = nn.ModuleDict()
        for aspect in self.aspects:
            self.aspect_heads[aspect] = nn.Sequential(
                nn.Linear(hidden_dim * 2 + 64 + hidden_dim * 2, 512),
                nn.LayerNorm(512),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(512, 256),
                nn.LayerNorm(256),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(256, 1)
            )
    
    def forward(self, sentence_embeddings, positions, aspect_idx):
        lstm_out, _ = self.shared_lstm(sentence_embeddings)
        
        aspect_emb_query = self.aspect_embedding(torch.tensor([aspect_idx], device=device))
        aspect_emb_query = aspect_emb_query.unsqueeze(1).expand(-1, lstm_out.size(1), -1)
        
        attended_out, _ = self.attention(aspect_emb_query, lstm_out, lstm_out)
        
        combined_lstm = lstm_out + attended_out
        pos_emb = self.position_embedding(positions)
        aspect_emb_concat = self.aspect_embedding(torch.tensor([aspect_idx], device=device))
        aspect_emb_concat = aspect_emb_concat.unsqueeze(1).expand(-1, sentence_embeddings.size(1), -1)
        
        combined = torch.cat([combined_lstm, pos_emb, aspect_emb_concat], dim=-1)
        
        aspect_name = self.aspects[aspect_idx]
        logits = self.aspect_heads[aspect_name](combined).squeeze(-1)
        
        return logits

print("✅ Policy Network defined (supports long documents)")


✅ Policy Network defined (supports long documents)


In [12]:
class ImprovedReferenceFreeReward:
    """Enhanced reward function with improved weights"""
    
    def __init__(self, encoder):
        self.encoder = encoder
        
    def compute_coherence(self, summary_sentences):
        if len(summary_sentences) < 2:
            return 1.0
        
        embeddings = self.encoder(summary_sentences)
        coherence_scores = []
        for i in range(len(embeddings) - 1):
            sim = torch.nn.functional.cosine_similarity(
                embeddings[i].unsqueeze(0), 
                embeddings[i+1].unsqueeze(0)
            )
            coherence_scores.append(sim.item())
        
        return np.mean(coherence_scores) if coherence_scores else 0.0
    
    def compute_coverage(self, summary_embeddings, document_embeddings):
        coverage_scores = []
        for doc_emb in document_embeddings:
            sims = torch.nn.functional.cosine_similarity(
                doc_emb.unsqueeze(0).expand(len(summary_embeddings), -1),
                summary_embeddings
            )
            coverage_scores.append(sims.max().item())
        
        return np.mean(coverage_scores)
    
    def compute_redundancy(self, summary_embeddings):
        if len(summary_embeddings) < 2:
            return 0.0
        
        redundancy_scores = []
        for i in range(len(summary_embeddings)):
            for j in range(i+1, len(summary_embeddings)):
                sim = torch.nn.functional.cosine_similarity(
                    summary_embeddings[i].unsqueeze(0),
                    summary_embeddings[j].unsqueeze(0)
                )
                redundancy_scores.append(sim.item())
        
        return np.mean(redundancy_scores) if redundancy_scores else 0.0
    
    def compute_diversity(self, summary_embeddings):
        if len(summary_embeddings) < 2:
            return 0.0
        
        distances = []
        for i in range(len(summary_embeddings)):
            for j in range(i+1, len(summary_embeddings)):
                dist = 1.0 - torch.nn.functional.cosine_similarity(
                    summary_embeddings[i].unsqueeze(0),
                    summary_embeddings[j].unsqueeze(0)
                ).item()
                distances.append(dist)
        
        return np.mean(distances) if distances else 0.0
    
    def compute_informativeness(self, summary_embeddings, document_embeddings):
        informativeness_scores = []
        for summ_emb in summary_embeddings:
            sims = torch.nn.functional.cosine_similarity(
                summ_emb.unsqueeze(0).expand(len(document_embeddings), -1),
                document_embeddings
            )
            informativeness_scores.append(sims.mean().item())
        
        return np.mean(informativeness_scores)
    
    def compute_position_bias(self, selected_indices, total_sentences):
        if len(selected_indices) == 0:
            return 0.0
        
        positions = [idx / max(total_sentences - 1, 1) for idx in selected_indices]
        
        has_beginning = any(p < 0.33 for p in positions)
        has_middle = any(0.33 <= p < 0.67 for p in positions)
        has_end = any(p >= 0.67 for p in positions)
        
        return (has_beginning + has_middle + has_end) / 3.0
    
    def compute_sentence_ordering_penalty(self, selected_indices):
        if len(selected_indices) < 2:
            return 1.0
        
        ordered = all(selected_indices[i] < selected_indices[i+1] 
                     for i in range(len(selected_indices)-1))
        
        if ordered:
            return 1.0
        else:
            inversions = 0
            for i in range(len(selected_indices)):
                for j in range(i+1, len(selected_indices)):
                    if selected_indices[i] > selected_indices[j]:
                        inversions += 1
            
            max_inversions = len(selected_indices) * (len(selected_indices) - 1) / 2
            return 1.0 - (inversions / max(max_inversions, 1))
    
    def compute_reward(self, selected_sentences, selected_indices, 
                      all_sentences, target_ratio, aspect):
        """IMPROVED reward with better weights"""
        if not selected_sentences or not all_sentences:
            return 0.0
        
        # Encode sentences
        summary_embeddings = self.encoder(selected_sentences)
        document_embeddings = self.encoder(all_sentences)
        
        # Component rewards
        coherence = self.compute_coherence(selected_sentences)
        coverage = self.compute_coverage(summary_embeddings, document_embeddings)
        redundancy = self.compute_redundancy(summary_embeddings)
        diversity = self.compute_diversity(summary_embeddings)
        informativeness = self.compute_informativeness(summary_embeddings, document_embeddings)
        position_bias = self.compute_position_bias(selected_indices, len(all_sentences))
        ordering_score = self.compute_sentence_ordering_penalty(selected_indices)
        
        # Length control
        gen_words = sum(len(s.split()) for s in selected_sentences)
        source_words = sum(len(s.split()) for s in all_sentences)
        compression_ratio = gen_words / max(source_words, 1)
        
        length_diff = abs(compression_ratio - target_ratio)
        length_reward = np.exp(-10 * length_diff)
        
        # IMPROVED WEIGHTS
        final_reward = (
            0.30 * coherence +           # Increased
            0.25 * coverage +            
            0.20 * (1 - redundancy) +    # Increased penalty
            0.15 * diversity +           # Increased
            0.05 * informativeness +     
            0.03 * position_bias +       
            0.02 * ordering_score        # New
        )
        
        final_reward = final_reward * (0.8 + 0.2 * length_reward)
        
        return final_reward

print("✅ Improved Reference-Free Reward Function defined")


✅ Improved Reference-Free Reward Function defined


In [13]:
class UnsupervisedRLAgent:
    """
    RL Agent with memory-efficient processing
    """
    def __init__(self, encoder, policy, learning_rate=1e-4, gamma=0.99, weight_decay=1e-5):
        self.encoder = encoder.to(device)
        self.policy = policy.to(device)
        
        self.optimizer = optim.Adam(self.policy.parameters(), 
                                    lr=learning_rate, 
                                    weight_decay=weight_decay)
        
        self.gamma = gamma
        self.aspects = ['facts', 'analysis', 'argument', 'judgement', 'statute']
        
        self.aspect_summary_ratios = {
            'facts': 0.12,
            'analysis': 0.12,
            'argument': 0.08,
            'judgement': 0.06,
            'statute': 0.08
        }
        
        self.min_summary_sentences = 3
        self.max_document_sentences = 500  # ADD THIS - Limit for GPU memory
        
        self.reward_function = ImprovedReferenceFreeReward(encoder)
        self.reward_baselines = {aspect: 0.0 for aspect in self.aspects}
        self.baseline_momentum = 0.9
        self.best_val_reward = 0.0
        self.patience = 15
        self.patience_counter = 0
    
    def preprocess_document(self, judgment_text):
        """Process sentences with memory limit"""
        sentences = sent_tokenize(judgment_text)
        sentences = [s.strip() for s in sentences if len(s.strip().split()) > 5]
        
        # LIMIT SENTENCES FOR GPU MEMORY
        if len(sentences) > self.max_document_sentences:
            print(f"   ⚠️ Document too long ({len(sentences)} sents), truncating to {self.max_document_sentences}", end='\r')
            sentences = sentences[:self.max_document_sentences]
        
        return sentences
    
    def encode_sentences(self, sentences):
        """Encode ALL sentences in batches"""
        if len(sentences) == 0:
            return torch.zeros(1, self.encoder.hidden_dim).to(device)
        
        batch_size = 16
        embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            emb = self.encoder(batch)
            embeddings.append(emb)
        
        return torch.cat(embeddings, dim=0)
    
    def select_sentences(self, logits, num_sentences, aspect, temperature=1.0):
        logits = logits / temperature
        probs = torch.sigmoid(logits)
        
        dist = torch.distributions.Bernoulli(probs)
        actions = dist.sample()
        log_probs = dist.log_prob(actions)
        
        num_selected = actions.sum().item()
        
        aspect_ratio = self.aspect_summary_ratios[aspect]
        max_sentences = max(self.min_summary_sentences, int(num_sentences * aspect_ratio))
        
        if num_selected < self.min_summary_sentences:
            top_k_indices = torch.topk(logits, k=self.min_summary_sentences).indices
            actions = torch.zeros_like(actions)
            actions[top_k_indices] = 1
        elif num_selected > max_sentences:
            selected_indices = (actions == 1).nonzero(as_tuple=True)[0]
            selected_logits = logits[selected_indices]
            top_k = torch.topk(selected_logits, k=max_sentences).indices
            keep_indices = selected_indices[top_k]
            actions = torch.zeros_like(actions)
            actions[keep_indices] = 1
        
        return actions, log_probs
    
    def train_step(self, judgment, temperature=1.0):
        """Train with memory management"""
        self.policy.train()
        
        sentences = self.preprocess_document(judgment)
        if len(sentences) < 3:
            return 0.0, {aspect: 0.0 for aspect in self.aspects}
        
        print(f"   Processing {len(sentences)} sentences...", end='\r')
        
        # Clear cache before processing
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        sentence_embeddings = self.encode_sentences(sentences)
        sentence_embeddings = sentence_embeddings.unsqueeze(0)
        
        positions = torch.arange(min(len(sentences), 1999), device=device).unsqueeze(0)
        if len(sentences) < positions.size(1):
            positions = positions[:, :len(sentences)]
        
        total_loss = 0.0
        aspect_rewards = {}
        
        for aspect_idx, aspect in enumerate(self.aspects):
            logits = self.policy(sentence_embeddings, positions, aspect_idx).squeeze(0)
            actions, log_probs = self.select_sentences(logits, len(sentences), aspect, temperature)
            
            selected_indices = (actions == 1).nonzero(as_tuple=True)[0].cpu().numpy()
            selected_sentences = [sentences[i] for i in selected_indices]
            
            reward = self.reward_function.compute_reward(
                selected_sentences, 
                selected_indices,
                sentences,
                self.aspect_summary_ratios[aspect],
                aspect
            )
            aspect_rewards[aspect] = reward
            
            self.reward_baselines[aspect] = (
                self.baseline_momentum * self.reward_baselines[aspect] + 
                (1 - self.baseline_momentum) * reward
            )
            
            advantage = reward - self.reward_baselines[aspect]
            loss = -(log_probs * actions * advantage).sum()
            
            probs = torch.sigmoid(logits)
            entropy = -(probs * torch.log(probs + 1e-10) + 
                       (1 - probs) * torch.log(1 - probs + 1e-10)).mean()
            loss = loss - 0.15 * entropy
            
            total_loss += loss
        
        total_loss = total_loss / len(self.aspects)
        self.optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy.parameters(), max_norm=1.0)
        self.optimizer.step()
        
        # Clear cache after update
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        return total_loss.item(), aspect_rewards
    
    def generate_summaries(self, judgment, temperature=0.3):
        """Generate summaries from FULL document"""
        self.policy.eval()
        
        with torch.no_grad():
            sentences = self.preprocess_document(judgment)
            if len(sentences) < 3:
                return {aspect: ' '.join(sentences) for aspect in self.aspects}
            
            # NO LIMIT - Process ALL sentences
            sentence_embeddings = self.encode_sentences(sentences)
            sentence_embeddings = sentence_embeddings.unsqueeze(0)
            
            positions = torch.arange(min(len(sentences), 1999), device=device).unsqueeze(0)
            if len(sentences) < positions.size(1):
                positions = positions[:, :len(sentences)]
            
            summaries = {}
            
            for aspect_idx, aspect in enumerate(self.aspects):
                logits = self.policy(sentence_embeddings, positions, aspect_idx).squeeze(0)
                
                aspect_ratio = self.aspect_summary_ratios[aspect]
                num_select = max(self.min_summary_sentences, 
                               int(len(sentences) * aspect_ratio))
                
                top_k_indices = torch.topk(logits, k=num_select).indices
                top_k_indices = sorted(top_k_indices.cpu().numpy())
                
                summary = ' '.join([sentences[i] for i in top_k_indices])
                summaries[aspect] = summary
            
            return summaries
    
    def evaluate(self, val_dataset):
        self.policy.eval()
        
        aspect_total_rewards = {aspect: 0.0 for aspect in self.aspects}
        
        for item in val_dataset:
            judgment = item['judgment']
            sentences = self.preprocess_document(judgment)
            generated_summaries = self.generate_summaries(judgment)
            
            for aspect_idx, aspect in enumerate(self.aspects):
                selected_sentences = sent_tokenize(generated_summaries[aspect])
                selected_indices = list(range(len(selected_sentences)))
                
                reward = self.reward_function.compute_reward(
                    selected_sentences,
                    selected_indices,
                    sentences,
                    self.aspect_summary_ratios[aspect],
                    aspect
                )
                aspect_total_rewards[aspect] += reward
        
        aspect_avg_rewards = {
            aspect: aspect_total_rewards[aspect] / len(val_dataset)
            for aspect in self.aspects
        }
        
        overall_avg = np.mean(list(aspect_avg_rewards.values()))
        
        return overall_avg, aspect_avg_rewards
    
    def check_early_stopping(self, val_reward):
        if val_reward > self.best_val_reward:
            self.best_val_reward = val_reward
            self.patience_counter = 0
            return False
        else:
            self.patience_counter += 1
            return self.patience_counter >= self.patience

print("✅ Unsupervised RL Agent defined (NO SENTENCE LIMITS)")


✅ Unsupervised RL Agent defined (NO SENTENCE LIMITS)


In [14]:
def train_unsupervised_rl(agent, train_dataset, val_dataset, num_epochs=50, 
                         save_path='final_inlegalbert_model.pt'):
    """Training with warmup - processes ALL documents fully"""
    training_history = {
        'train_loss': [],
        'train_rewards': {aspect: [] for aspect in agent.aspects},
        'val_rewards': {aspect: [] for aspect in agent.aspects},
        'overall_val_reward': [],
        'learning_rate': []
    }
    
    warmup_epochs = 3
    base_lr = agent.optimizer.param_groups[0]['lr']
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        agent.optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )
    
    for epoch in range(num_epochs):
        # Warmup
        if epoch < warmup_epochs:
            warmup_lr = base_lr * (epoch + 1) / warmup_epochs
            for param_group in agent.optimizer.param_groups:
                param_group['lr'] = warmup_lr
            print(f"   🔥 Warmup: LR = {warmup_lr:.6f}")
        
        temperature = max(0.3, 1.0 - epoch / num_epochs)
        
        epoch_losses = []
        epoch_aspect_rewards = {aspect: [] for aspect in agent.aspects}
        
        pbar = tqdm(train_dataset, desc=f'Epoch {epoch+1}/{num_epochs}')
        for item in pbar:
            judgment = item['judgment']
            
            loss, aspect_rewards = agent.train_step(judgment, temperature)
            
            epoch_losses.append(loss)
            for aspect in agent.aspects:
                if aspect_rewards[aspect] > 0:
                    epoch_aspect_rewards[aspect].append(aspect_rewards[aspect])
            
            avg_reward = np.mean([r for r in aspect_rewards.values() if r > 0]) if any(aspect_rewards.values()) else 0
            pbar.set_postfix({
                'loss': f'{loss:.4f}', 
                'reward': f'{avg_reward:.4f}',
                'temp': f'{temperature:.2f}'
            })
        
        avg_train_loss = np.mean(epoch_losses) if epoch_losses else 0
        
        for aspect in agent.aspects:
            if epoch_aspect_rewards[aspect]:
                training_history['train_rewards'][aspect].append(np.mean(epoch_aspect_rewards[aspect]))
            else:
                training_history['train_rewards'][aspect].append(0.0)
        
        # Validation
        overall_val_reward, aspect_val_rewards = agent.evaluate(val_dataset)
        
        if epoch >= warmup_epochs:
            scheduler.step(overall_val_reward)
        
        training_history['train_loss'].append(avg_train_loss)
        training_history['overall_val_reward'].append(overall_val_reward)
        training_history['learning_rate'].append(agent.optimizer.param_groups[0]['lr'])
        for aspect in agent.aspects:
            training_history['val_rewards'][aspect].append(aspect_val_rewards[aspect])
        
        print(f'\n{"="*70}')
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'{"="*70}')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Overall Val Reward: {overall_val_reward:.4f}')
        print(f'  Learning Rate: {agent.optimizer.param_groups[0]["lr"]:.6f}')
        print(f'  Temperature: {temperature:.4f}')
        print(f'\n  Aspect-wise Val Rewards:')
        for aspect in agent.aspects:
            print(f'    {aspect:12s}: {aspect_val_rewards[aspect]:.4f}')
        
        # Save best
        if overall_val_reward > agent.best_val_reward:
            agent.best_val_reward = overall_val_reward
            agent.patience_counter = 0
            torch.save({
                'epoch': epoch,
                'policy_state_dict': agent.policy.state_dict(),
                'optimizer_state_dict': agent.optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_reward': overall_val_reward,
                'aspect_val_rewards': aspect_val_rewards,
                'training_history': training_history,
            }, save_path)
            print(f'\n  ✅ Saved best model (Val Reward: {overall_val_reward:.4f})')
        
        # Early stopping
        if agent.check_early_stopping(overall_val_reward):
            print(f'\n⚠️ Early stopping at epoch {epoch+1}')
            print(f'  Best val reward: {agent.best_val_reward:.4f}')
            break
        
        print(f'{"="*70}\n')
    
    return training_history

print("✅ Training function defined with warmup")


✅ Training function defined with warmup


In [ ]:
print("\n" + "="*70)
print("🚀 INITIALIZING INLEGALBERT MODEL")
print("="*70)
print(f"   Dataset: {len(dataset)} Supreme Court judgments")
print(f"   Training: {len(train_dataset)} documents")
print(f"   Validation: {len(val_dataset)} documents")
print(f"   Test: {len(test_dataset)} documents")
print(f"   🔥 NO LIMITS - Processing ALL sentences in ALL documents")
print("="*70 + "\n")

encoder = SentenceEncoder(model_name='law-ai/InLegalBERT', hidden_dim=768)

policy = MultiAspectPolicyNetwork(
    input_dim=768,
    hidden_dim=256, 
    num_aspects=5, 
    dropout=0.5
)

agent = UnsupervisedRLAgent(
    encoder=encoder,
    policy=policy,
    learning_rate=1e-4,
    weight_decay=1e-5
)

print(f"\n📊 Model Statistics:")
print(f"   Encoder: InLegalBERT (768-dim, frozen)")
print(f"   Total params: {sum(p.numel() for p in policy.parameters()):,}")
print(f"   Trainable: {sum(p.numel() for p in policy.parameters() if p.requires_grad):,}")

print("\n" + "="*70)
print("🎯 STARTING TRAINING")
print("="*70)
print("⚡ Improvements:")
print("   • InLegalBERT (Indian legal domain)")
print("   • Enhanced rewards (coherence↑30%, redundancy↓20%, diversity↑15%)")
print("   • Sentence ordering penalty")
print("   • Patience: 15 epochs")
print("   • Learning rate warmup: 3 epochs")
print("   • NO SENTENCE LIMITS - Full document processing")
print("="*70 + "\n")

history = train_unsupervised_rl(
    agent=agent,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    num_epochs=10,
    save_path='final_inlegalbert_model.pt'
)

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)



🚀 INITIALIZING INLEGALBERT MODEL
   Dataset: 400 Supreme Court judgments
   Training: 280 documents
   Validation: 60 documents
   Test: 60 documents
   🔥 NO LIMITS - Processing ALL sentences in ALL documents

🔄 Loading law-ai/InLegalBERT...


2025-11-14 17:06:23.552541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763139983.570983     142 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763139983.576018     142 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Epoch 8/10:  50%|████▉     | 139/280 [39:50<42:36, 18.13s/it, loss=-0.1043, reward=0.5870, temp=0.30]

Epoch 8/10:  50%|█████     | 140/280 [39:52<30:58, 13.27s/it, loss=-0.1063, reward=0.5794, temp=0.30]

Epoch 8/10:  50%|█████     | 141/280 [39:57<25:08, 10.85s/it, loss=-0.1009, reward=0.5989, temp=0.30]

Epoch 8/10:  51%|█████     | 142/280 [40:02<20:37,  8.97s/it, loss=-0.1071, reward=0.5913, temp=0.30]

Epoch 8/10:  51%|█████     | 143/280 [40:10<19:32,  8.56s/it, loss=-0.0995, reward=0.6011, temp=0.30]

In [1]:
def load_trained_model(checkpoint_path='final_inlegalbert_model.pt'):
    """Load the trained model from checkpoint"""
    print("\n" + "="*70)
    print("📂 LOADING TRAINED MODEL")
    print("="*70)
    
    encoder = SentenceEncoder(model_name='law-ai/InLegalBERT', hidden_dim=768)
    policy = MultiAspectPolicyNetwork(
        input_dim=768, 
        hidden_dim=256, 
        num_aspects=5, 
        dropout=0.5
    )
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    policy.load_state_dict(checkpoint['policy_state_dict'])
    
    agent = UnsupervisedRLAgent(
        encoder=encoder,
        policy=policy,
        learning_rate=1e-4,
        weight_decay=1e-5
    )
    
    agent.policy.load_state_dict(checkpoint['policy_state_dict'])
    agent.policy.eval()
    
    print(f"\n✅ Model loaded successfully!")
    print(f"   Training completed at epoch: {checkpoint['epoch']}")
    print(f"   Best validation reward: {checkpoint['val_reward']:.4f}")
    print(f"\n   Aspect-wise Validation Rewards:")
    for aspect, reward in checkpoint['aspect_val_rewards'].items():
        print(f"     {aspect:12s}: {reward:.4f}")
    print("="*70 + "\n")
    
    return agent, checkpoint

# Load the trained model
agent, checkpoint = load_trained_model('final_inlegalbert_model.pt')



📂 LOADING TRAINED MODEL


NameError: name 'SentenceEncoder' is not defined

In [ ]:
def print_test_sample_summaries(agent, test_dataset, num_samples=5):
    """
    Generate and print sample summaries from TEST SET (unseen data)
    """
    print("\n" + "="*80)
    print(f"📝 SAMPLE SUMMARIES FROM TEST SET (UNSEEN DATA)")
    print(f"   Total test documents: {len(test_dataset)}")
    print("="*80 + "\n")
    
    # Random samples from test set
    sample_indices = random.sample(range(len(test_dataset)), min(num_samples, len(test_dataset)))
    
    for idx, sample_idx in enumerate(sample_indices, 1):
        item = test_dataset[sample_idx]
        judgment = item['judgment']
        judgment_file = item['judgment_file']
        
        print(f"\n{'='*80}")
        print(f"TEST SAMPLE {idx}/{num_samples}: {judgment_file}")
        print(f"{'='*80}\n")
        
        # Original document info
        sentences = agent.preprocess_document(judgment)
        original_words = sum(len(s.split()) for s in sentences)
        
        print(f"📄 ORIGINAL DOCUMENT:")
        print(f"   Total sentences: {len(sentences)}")
        print(f"   Total words: {original_words:,}")
        print(f"\n   First 3 sentences:")
        for i, sent in enumerate(sentences[:3], 1):
            truncated = sent[:250] + "..." if len(sent) > 250 else sent
            print(f"   {i}. {truncated}")
        
        # Generate summaries
        print(f"\n{'─'*80}")
        print(f"🤖 GENERATED SUMMARIES:")
        print(f"{'─'*80}\n")
        
        summaries = agent.generate_summaries(judgment)
        
        # Compute quality metrics for each aspect
        for aspect in agent.aspects:
            summary = summaries[aspect]
            summary_sentences = sent_tokenize(summary)
            word_count = len(summary.split())
            
            # Compute quality metrics
            if summary_sentences and sentences:
                summary_embeddings = agent.encoder(summary_sentences)
                document_embeddings = agent.encoder(sentences)
                
                coherence = agent.reward_function.compute_coherence(summary_sentences)
                coverage = agent.reward_function.compute_coverage(summary_embeddings, document_embeddings)
                redundancy = agent.reward_function.compute_redundancy(summary_embeddings)
                diversity = agent.reward_function.compute_diversity(summary_embeddings)
                
                print(f"📌 {aspect.upper()}:")
                print(f"   Length: {len(summary_sentences)} sentences | {word_count} words")
                print(f"   Quality Metrics:")
                print(f"     • Coherence:      {coherence:.3f}")
                print(f"     • Coverage:       {coverage:.3f}")
                print(f"     • Redundancy:     {redundancy:.3f} (lower is better)")
                print(f"     • Diversity:      {diversity:.3f}")
                print(f"\n   Summary Text:")
                for i, sent in enumerate(summary_sentences, 1):
                    print(f"     {i}. {sent}")
                print()
        
        # Overall compression statistics
        total_summary_words = sum(len(summaries[a].split()) for a in agent.aspects)
        overall_compression = total_summary_words / original_words
        
        print(f"{'─'*80}")
        print(f"📊 COMPRESSION STATISTICS:")
        print(f"{'─'*80}")
        print(f"   Original Document:     {original_words:6,} words")
        print(f"   Combined Summary:      {total_summary_words:6,} words")
        print(f"   Overall Compression:   {overall_compression:7.2%}")
        print(f"\n   Aspect-wise Breakdown:")
        for aspect in agent.aspects:
            aspect_words = len(summaries[aspect].split())
            aspect_ratio = aspect_words / original_words
            print(f"     {aspect:12s}: {aspect_words:5,} words ({aspect_ratio:6.2%})")
        
        print(f"\n{'='*80}\n")

# Generate and print sample summaries from test set
print_test_sample_summaries(agent, test_dataset, num_samples=5)


In [ ]:
def evaluate_test_set_metrics(agent, test_dataset):
    """
    Compute comprehensive reference-free metrics on TEST SET
    """
    print("\n" + "="*70)
    print("📊 COMPUTING REFERENCE-FREE METRICS ON TEST SET")
    print("="*70 + "\n")
    
    aspect_metrics = {aspect: [] for aspect in agent.aspects}
    
    for item in tqdm(test_dataset, desc="Evaluating test set"):
        judgment = item['judgment']
        sentences = agent.preprocess_document(judgment)
        generated_summaries = agent.generate_summaries(judgment)
        
        for aspect in agent.aspects:
            summary = generated_summaries[aspect]
            summary_sentences = sent_tokenize(summary)
            
            if not summary_sentences or not sentences:
                continue
            
            # Encode for semantic analysis
            summary_embeddings = agent.encoder(summary_sentences)
            document_embeddings = agent.encoder(sentences)
            
            # Find approximate selected indices
            selected_indices = []
            for summ_sent in summary_sentences:
                for idx, orig_sent in enumerate(sentences):
                    if summ_sent.strip()[:50] in orig_sent or orig_sent[:50] in summ_sent.strip():
                        selected_indices.append(idx)
                        break
            
            if not selected_indices:
                selected_indices = list(range(len(summary_sentences)))
            
            # Compute all reference-free metrics
            coherence = agent.reward_function.compute_coherence(summary_sentences)
            coverage = agent.reward_function.compute_coverage(summary_embeddings, document_embeddings)
            redundancy = agent.reward_function.compute_redundancy(summary_embeddings)
            diversity = agent.reward_function.compute_diversity(summary_embeddings)
            informativeness = agent.reward_function.compute_informativeness(summary_embeddings, document_embeddings)
            position_bias = agent.reward_function.compute_position_bias(selected_indices, len(sentences))
            ordering_score = agent.reward_function.compute_sentence_ordering_penalty(selected_indices)
            
            # Length metrics
            gen_words = len(summary.split())
            source_words = sum(len(s.split()) for s in sentences)
            compression_ratio = gen_words / max(source_words, 1)
            
            # Total reward
            total_reward = agent.reward_function.compute_reward(
                summary_sentences, selected_indices, sentences,
                agent.aspect_summary_ratios[aspect], aspect
            )
            
            metrics = {
                'total_reward': total_reward,
                'coherence': coherence,
                'coverage': coverage,
                'redundancy': redundancy,
                'diversity': diversity,
                'informativeness': informativeness,
                'position_bias': position_bias,
                'ordering_score': ordering_score,
                'compression_ratio': compression_ratio,
                'summary_words': gen_words,
                'source_words': source_words,
                'summary_sentences': len(summary_sentences),
                'source_sentences': len(sentences)
            }
            
            aspect_metrics[aspect].append(metrics)
    
    # Print comprehensive summary
    print("\n" + "="*70)
    print("📈 TEST SET RESULTS (FINAL EVALUATION)")
    print("="*70 + "\n")
    
    for aspect in agent.aspects:
        if aspect_metrics[aspect]:
            avg_metrics = {
                k: np.mean([m[k] for m in aspect_metrics[aspect]])
                for k in aspect_metrics[aspect][0].keys()
            }
            
            print(f"📌 {aspect.upper()}")
            print(f"   {'─'*60}")
            print(f"   Total Reward:      {avg_metrics['total_reward']:.4f}")
            print(f"   ├─ Coherence:      {avg_metrics['coherence']:.4f}  (semantic flow)")
            print(f"   ├─ Coverage:       {avg_metrics['coverage']:.4f}  (content coverage)")
            print(f"   ├─ Diversity:      {avg_metrics['diversity']:.4f}  (information variety)")
            print(f"   ├─ Informativeness:{avg_metrics['informativeness']:.4f}  (importance)")
            print(f"   ├─ Redundancy:     {avg_metrics['redundancy']:.4f}  (lower = better)")
            print(f"   ├─ Position Bias:  {avg_metrics['position_bias']:.4f}  (doc coverage)")
            print(f"   └─ Ordering Score: {avg_metrics['ordering_score']:.4f}  (sentence order)")
            print(f"\n   Length Statistics:")
            print(f"     Compression:     {avg_metrics['compression_ratio']:.2%}")
            print(f"     Avg Summary:     {avg_metrics['summary_words']:.0f} words, {avg_metrics['summary_sentences']:.0f} sents")
            print(f"     Avg Source:      {avg_metrics['source_words']:.0f} words, {avg_metrics['source_sentences']:.0f} sents")
            print()
    
    return aspect_metrics

# Compute metrics on test set
test_metrics = evaluate_test_set_metrics(agent, test_dataset)


In [ ]:
def export_test_metrics_to_csv(test_metrics, filename='test_set_metrics.csv'):
    """
    Export test set metrics to CSV with detailed statistics
    """
    print(f"\n💾 Exporting test metrics to {filename}...")
    
    data = []
    for aspect in agent.aspects:
        for idx, metrics in enumerate(test_metrics[aspect]):
            row = {'sample_id': idx, 'aspect': aspect, **metrics}
            data.append(row)
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    
    print(f"✅ Saved {len(df)} test metric records\n")
    
    # Detailed statistical summary
    print("="*90)
    print("📊 DETAILED TEST SET STATISTICS BY ASPECT")
    print("="*90 + "\n")
    
    summary = df.groupby('aspect')[['total_reward', 'coherence', 'coverage', 
                                     'diversity', 'informativeness', 'redundancy',
                                     'ordering_score', 'position_bias',
                                     'compression_ratio']].agg(['mean', 'std', 'min', 'max'])
    
    print(summary.round(4))
    print("\n" + "="*90 + "\n")
    
    # Overall statistics across all aspects
    print("📈 OVERALL TEST SET STATISTICS (All Aspects Combined):")
    print("="*90)
    overall_stats = df[['total_reward', 'coherence', 'coverage', 'diversity', 
                        'redundancy', 'compression_ratio']].describe()
    print(overall_stats.round(4))
    print("="*90 + "\n")
    
    return df

# Export test metrics
test_df = export_test_metrics_to_csv(test_metrics)


In [ ]:
def create_test_set_visualizations(test_metrics, test_df):
    """
    Create comprehensive visualizations for test set evaluation
    """
    print("\n📊 Generating comprehensive test set visualizations...\n")
    
    aspects = agent.aspects
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
    
    # Aggregate metrics
    agg_data = {}
    for aspect in aspects:
        if test_metrics[aspect]:
            agg_data[aspect] = {
                k: np.mean([m[k] for m in test_metrics[aspect]])
                for k in test_metrics[aspect][0].keys()
            }
    
    # Create comprehensive figure
    fig = plt.figure(figsize=(20, 14))
    gs = fig.add_gridspec(4, 3, hspace=0.35, wspace=0.3)
    
    # 1. Total Reward by Aspect
    ax1 = fig.add_subplot(gs[0, 0])
    total_rewards = [agg_data[a]['total_reward'] for a in aspects]
    bars = ax1.bar(aspects, total_rewards, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax1.set_title('Total Reward by Aspect (Test Set)', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Reward Score')
    ax1.set_xticklabels(aspects, rotation=45, ha='right')
    ax1.grid(True, alpha=0.3, axis='y')
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 2. Quality Metrics Comparison
    ax2 = fig.add_subplot(gs[0, 1])
    metrics_plot = ['coherence', 'coverage', 'diversity', 'informativeness']
    df_plot = pd.DataFrame({
        a: [agg_data[a][m] for m in metrics_plot] for a in aspects
    }, index=metrics_plot)
    df_plot.T.plot(kind='bar', ax=ax2, alpha=0.8, width=0.8)
    ax2.set_title('Quality Metrics by Aspect', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Score')
    ax2.legend(title='Metrics', fontsize=8, loc='upper right')
    ax2.set_xticklabels(aspects, rotation=45, ha='right')
    ax2.grid(True, alpha=0.3)
    
    # 3. Coherence vs Coverage Scatter
    ax3 = fig.add_subplot(gs[0, 2])
    for idx, aspect in enumerate(aspects):
        ax3.scatter(agg_data[aspect]['coherence'], agg_data[aspect]['coverage'],
                   label=aspect, color=colors[idx], s=250, alpha=0.7,
                   edgecolor='black', linewidth=1.5)
    ax3.set_title('Coherence vs Coverage', fontweight='bold', fontsize=12)
    ax3.set_xlabel('Coherence Score')
    ax3.set_ylabel('Coverage Score')
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    ax3.plot([0, 1], [0, 1], 'k--', alpha=0.3)
    
    # 4. Redundancy by Aspect
    ax4 = fig.add_subplot(gs[1, 0])
    redundancy = [agg_data[a]['redundancy'] for a in aspects]
    bars = ax4.bar(aspects, redundancy, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax4.set_title('Redundancy by Aspect (Lower = Better)', fontweight='bold', fontsize=12)
    ax4.set_ylabel('Redundancy Score')
    ax4.set_xticklabels(aspects, rotation=45, ha='right')
    ax4.axhline(y=0.2, color='red', linestyle='--', alpha=0.5, label='Target < 0.20')
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    for bar in bars:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 5. Compression Ratios
    ax5 = fig.add_subplot(gs[1, 1])
    compression = [agg_data[a]['compression_ratio'] for a in aspects]
    bars = ax5.bar(aspects, compression, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax5.set_title('Compression Ratios by Aspect', fontweight='bold', fontsize=12)
    ax5.set_ylabel('Compression Ratio')
    ax5.set_xticklabels(aspects, rotation=45, ha='right')
    ax5.grid(True, alpha=0.3, axis='y')
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1%}', ha='center', va='bottom', fontsize=9)
    
    # 6. Ordering Score
    ax6 = fig.add_subplot(gs[1, 2])
    ordering = [agg_data[a]['ordering_score'] for a in aspects]
    bars = ax6.bar(aspects, ordering, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax6.set_title('Sentence Ordering Score', fontweight='bold', fontsize=12)
    ax6.set_ylabel('Ordering Score (1.0 = Perfect)')
    ax6.set_xticklabels(aspects, rotation=45, ha='right')
    ax6.grid(True, alpha=0.3, axis='y')
    for bar in bars:
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 7. Reward Distribution (Box Plot)
    ax7 = fig.add_subplot(gs[2, 0])
    reward_data = [test_df[test_df['aspect'] == a]['total_reward'].values for a in aspects]
    bp = ax7.boxplot(reward_data, labels=aspects, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    ax7.set_title('Reward Distribution by Aspect', fontweight='bold', fontsize=12)
    ax7.set_ylabel('Total Reward')
    ax7.set_xticklabels(aspects, rotation=45, ha='right')
    ax7.grid(True, alpha=0.3, axis='y')
    
    # 8. Correlation Heatmap
    ax8 = fig.add_subplot(gs[2, 1])
    corr_cols = ['coherence', 'coverage', 'diversity', 'informativeness', 'redundancy']
    corr_data = test_df[corr_cols].corr()
    sns.heatmap(corr_data, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                ax=ax8, cbar_kws={'label': 'Correlation'}, linewidths=1, vmin=-1, vmax=1)
    ax8.set_title('Metric Correlations', fontweight='bold', fontsize=12)
    
    # 9. Summary Length Distribution
    ax9 = fig.add_subplot(gs[2, 2])
    for idx, aspect in enumerate(aspects):
        aspect_data = test_df[test_df['aspect'] == aspect]['summary_words'].values
        ax9.hist(aspect_data, alpha=0.6, label=aspect, color=colors[idx], 
                bins=10, edgecolor='black', linewidth=0.5)
    ax9.set_title('Summary Word Count Distribution', fontweight='bold', fontsize=12)
    ax9.set_xlabel('Number of Words')
    ax9.set_ylabel('Frequency')
    ax9.legend(fontsize=9)
    ax9.grid(True, alpha=0.3)
    
    # 10. Position Bias
    ax10 = fig.add_subplot(gs[3, 0])
    position_bias = [agg_data[a]['position_bias'] for a in aspects]
    bars = ax10.bar(aspects, position_bias, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax10.set_title('Position Bias (Document Coverage)', fontweight='bold', fontsize=12)
    ax10.set_ylabel('Position Score')
    ax10.set_xticklabels(aspects, rotation=45, ha='right')
    ax10.grid(True, alpha=0.3, axis='y')
    ax10.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='Perfect (1.0)')
    ax10.legend(fontsize=9)
    
    # 11. Informativeness
    ax11 = fig.add_subplot(gs[3, 1])
    informativeness = [agg_data[a]['informativeness'] for a in aspects]
    bars = ax11.bar(aspects, informativeness, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax11.set_title('Informativeness by Aspect', fontweight='bold', fontsize=12)
    ax11.set_ylabel('Informativeness Score')
    ax11.set_xticklabels(aspects, rotation=45, ha='right')
    ax11.grid(True, alpha=0.3, axis='y')
    
    # 12. Overall Comparison (Radar Chart Style)
    ax12 = fig.add_subplot(gs[3, 2])
    metrics_compare = ['coherence', 'coverage', 'diversity', 'informativeness']
    x_pos = np.arange(len(metrics_compare))
    width = 0.15
    for idx, aspect in enumerate(aspects):
        values = [agg_data[aspect][m] for m in metrics_compare]
        ax12.bar(x_pos + idx*width, values, width, label=aspect, 
                color=colors[idx], alpha=0.8, edgecolor='black', linewidth=0.5)
    ax12.set_title('Overall Metric Comparison', fontweight='bold', fontsize=12)
    ax12.set_ylabel('Score')
    ax12.set_xticks(x_pos + width * 2)
    ax12.set_xticklabels(metrics_compare, rotation=45, ha='right')
    ax12.legend(fontsize=8)
    ax12.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle('InLegalBERT Model - Test Set Comprehensive Evaluation\n(Reference-Free Metrics on Unseen Data)', 
                 fontsize=16, fontweight='bold', y=0.998)
    
    plt.savefig('test_set_comprehensive_evaluation.png', dpi=300, bbox_inches='tight')
    print("✅ Saved visualization to 'test_set_comprehensive_evaluation.png'\n")
    plt.show()

# Create visualizations
create_test_set_visualizations(test_metrics, test_df)


In [ ]:
def print_final_summary_table(test_metrics):
    """
    Print a comprehensive summary table of test results
    """
    print("\n" + "="*90)
    print("📋 FINAL TEST SET EVALUATION SUMMARY")
    print("="*90 + "\n")
    
    # Calculate averages for each aspect
    print(f"{'Aspect':<12} {'Reward':>8} {'Coher':>7} {'Cover':>7} {'Redun':>7} {'Diver':>7} {'Order':>7} {'Compr':>7}")
    print("="*90)
    
    overall_rewards = []
    
    for aspect in agent.aspects:
        if test_metrics[aspect]:
            avg = {k: np.mean([m[k] for m in test_metrics[aspect]]) 
                  for k in test_metrics[aspect][0].keys()}
            
            overall_rewards.append(avg['total_reward'])
            
            print(f"{aspect:<12} "
                  f"{avg['total_reward']:>8.4f} "
                  f"{avg['coherence']:>7.3f} "
                  f"{avg['coverage']:>7.3f} "
                  f"{avg['redundancy']:>7.3f} "
                  f"{avg['diversity']:>7.3f} "
                  f"{avg['ordering_score']:>7.3f} "
                  f"{avg['compression_ratio']:>6.1%}")
    
    print("="*90)
    print(f"{'AVERAGE':<12} {np.mean(overall_rewards):>8.4f}")
    print("="*90 + "\n")
    
    # Key findings
    print("🎯 KEY FINDINGS:")
    print("-" * 90)
    
    all_coherence = [np.mean([m['coherence'] for m in test_metrics[a]]) for a in agent.aspects if test_metrics[a]]
    all_redundancy = [np.mean([m['redundancy'] for m in test_metrics[a]]) for a in agent.aspects if test_metrics[a]]
    all_coverage = [np.mean([m['coverage'] for m in test_metrics[a]]) for a in agent.aspects if test_metrics[a]]
    
    print(f"  • Average Coherence:      {np.mean(all_coherence):.4f}")
    print(f"  • Average Coverage:       {np.mean(all_coverage):.4f}")
    print(f"  • Average Redundancy:     {np.mean(all_redundancy):.4f} (target: < 0.20)")
    print(f"  • Best Aspect:            {agent.aspects[np.argmax(overall_rewards)].upper()} "
          f"(reward: {max(overall_rewards):.4f})")
    print(f"  • Total Test Documents:   {len(test_dataset)}")
    print(f"\n  ✅ Model successfully evaluated on UNSEEN test data!")
    print("="*90 + "\n")

# Print final summary
print_final_summary_table(test_metrics)


In [ ]:
def export_test_summaries_to_file(agent, test_dataset, num_samples=10, 
                                   output_file='test_set_sample_summaries.txt'):
    """
    Export sample summaries from test set to text file
    """
    print(f"\n💾 Exporting {num_samples} test sample summaries to {output_file}...")
    
    sample_indices = random.sample(range(len(test_dataset)), min(num_samples, len(test_dataset)))
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("INLEGALBERT MODEL - TEST SET SAMPLE SUMMARIES\n")
        f.write("Reference-Free Unsupervised RL Training\n")
        f.write(f"Test Set Size: {len(test_dataset)} documents\n")
        f.write("="*80 + "\n\n")
        
        for idx, sample_idx in enumerate(sample_indices, 1):
            item = test_dataset[sample_idx]
            judgment = item['judgment']
            judgment_file = item['judgment_file']
            
            sentences = agent.preprocess_document(judgment)
            summaries = agent.generate_summaries(judgment)
            
            original_words = sum(len(s.split()) for s in sentences)
            total_summary_words = sum(len(summaries[a].split()) for a in agent.aspects)
            
            f.write(f"\n{'='*80}\n")
            f.write(f"TEST SAMPLE {idx}: {judgment_file}\n")
            f.write(f"{'='*80}\n\n")
            f.write(f"Original: {len(sentences)} sentences, {original_words:,} words\n")
            f.write(f"Summary: {total_summary_words:,} words ({total_summary_words/original_words:.2%} compression)\n\n")
            
            for aspect in agent.aspects:
                f.write(f"\n{aspect.upper()}:\n")
                f.write(f"{'-'*80}\n")
                f.write(f"{summaries[aspect]}\n")
            
            f.write("\n")
    
    print(f"✅ Exported {num_samples} test summaries to {output_file}\n")

# Export test summaries
export_test_summaries_to_file(agent, test_dataset, num_samples=10)
